# https://spark.apache.org/docs/3.1.1/api/python/reference/api/pyspark.sql.functions.aggregate.html

In [7]:
# Intialize spark session
from pyspark.sql import SparkSession

spark = (
    SparkSession 
    .builder 
    .appName("pyspark_study_spark_session") 
    .config("spark.some.config.option", "some-value") 
    .getOrCreate()
)

23/11/08 19:02:32 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


In [23]:
df = spark.createDataFrame([(1, [20.0, 4.0, 2.0, 6.0, 10.0])], ("id", "values"))
df.show()

+---+--------------------+
| id|              values|
+---+--------------------+
|  1|[20.0, 4.0, 2.0, ...|
+---+--------------------+



In [32]:
df.select(
    F.aggregate(
        col=F.col("values"), 
        initialValue=F.lit(0.0), 
        merge=lambda acc, x: acc + x # acc: accumulator, x: current value
    )
    .alias("sum")
).show()

+----+
| sum|
+----+
|42.0|
+----+



In [38]:
def merge(acc, x):
    count = acc.count + 1
    sum = acc.sum + x
    return F.struct(count.alias("count"), sum.alias("sum"))

In [44]:
df.select(
    F.aggregate(
        "values",
        F.struct(F.lit(0).alias("count"), F.lit(0.0).alias("sum")),
        merge,
        # lambda acc: acc.sum / acc.count,
    ).alias("mean")
).show()

+---------+
|     mean|
+---------+
|{5, 42.0}|
+---------+



In [41]:
F.struct(F.lit(0).alias("count"), F.lit(0.0).alias("sum"))

Column<'struct(0 AS count, 0.0 AS sum)'>